## Processing of STARR-seq reads from 500 bp library

In [1]:
cd /staging/leuven/stg_00002/lcb/lcb_projects/CSE/500bp_ATAC_peak_library/counting

### 01.fastqc

In [4]:
module load FastQC/0.11.8-Java-1.8.0_162
cat STARRseq_names.txt | while read line
do
NovaSeq6000_200123="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200123/Demultiplexed/${line%STARR*}*R2*"
NovaSeq6000_200416="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200416/Demultiplexed/${line%STARR*}*R2*"
fastqc -t 10 ${NovaSeq6000_200123} ${NovaSeq6000_200416} -o 01.fastqc
done

Skipping '/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200123/Demultiplexed/STA__073ce2__MM029_*R2*' which didn't exist, or couldn't be read
Started analysis of STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 5% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 10% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 15% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 20% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 25% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 30% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 35% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 40% complete for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Approx 45% complete for STA__073ce2__

### 02.clean_reads

In [5]:
module use /data/leuven/software/biomed/skylake_centos7/2018a/modules/all/
module load SeqKit/0.10.2
module load cutadapt/1.18-foss-2018a-Python-3.7.0
start=`date +%s`
cat STARRseq_names.txt | while read line
do
NovaSeq6000_200123="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200123/Demultiplexed/${line%STARR*}*R2*"
NovaSeq6000_200416="/staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200416/Demultiplexed/${line%STARR*}*R2*"
    # Do cutadapt trimming on R2 reads and then reverse complement
cutadapt -g CTTATCATGTCTGCTCGAAGCGCACTGAGCCTCCACCTAGCCT -e 0.15 -m 7 -M 7 -j 10 --discard-untrimmed \
         ${NovaSeq6000_200416} | \
seqkit seq -r -p -o 02.clean_reads/${line}_R2.fastq.gz

done
end=`date +%s`
echo $((end-start))

This is cutadapt 1.18 with Python 3.7.0
Command line parameters: -g CTTATCATGTCTGCTCGAAGCGCACTGAGCCTCCACCTAGCCT -e 0.15 -m 7 -M 7 -j 10 --discard-untrimmed /staging/leuven/stg_00002/lcb/ngs_runs/NovaSeq6000_20200416/Demultiplexed/STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_S16_R2_001.fastq.gz
Processing reads on 10 cores in single-end mode ...
Finished in 43.34 s (4 us/read; 16.28 M reads/minute).

=== Summary ===

Total reads processed:              11,762,241
Reads with adapters:                10,665,592 (90.7%)
Reads that were too short:             366,781 (3.1%)
Reads that were too long:            1,377,004 (11.7%)
Reads written (passing filters):    10,018,456 (85.2%)

Total basepairs processed:   588,112,050 bp
Total written (filtered):     70,129,192 bp (11.9%)

=== Adapter 1 ===

Sequence: CTTATCATGTCTGCTCGAAGCGCACTGAGCCTCCACCTAGCCT; Type: regular 5'; Length: 43; Trimmed: 10665592 times.

No. of allowed errors:
0-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20-25 bp: 3; 26-32 bp: 4;

In [6]:
module load fastp/0.20.0-foss-2018a
cat STARRseq_names.txt | while read line
do
# Filter for Q30 reads
fastp -i 02.clean_reads/${line}_R2.fastq.gz -L -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 -o 02.clean_reads/${line}_q30.fastq.gz
echo -e "Number of reads after Q30 filtering for ${line}: " $(( $(zcat 02.clean_reads/${line}_q30.fastq.gz | wc -l) / 4 ))
done

Detecting adapter sequence for read1...
No adapter detected for read1

Read1 before filtering:
total reads: 10018456
total bases: 70129192
Q20 bases: 60498494(86.2672%)
Q30 bases: 49862678(71.1012%)

Read1 after filtering:
total reads: 6491053
total bases: 45437371
Q20 bases: 43266174(95.2216%)
Q30 bases: 37538162(82.6152%)

Filtering result:
reads passed filter: 6491053
reads failed due to low quality: 3527403
reads failed due to too many N: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 0%

JSON report: /dev/null/fastp.json
HTML report: /dev/null/fastp.html

fastp -i 02.clean_reads/STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_R2.fastq.gz -L -e 30 -h /dev/null/fastp.html -j /dev/null/fastp.json -w 8 -o 02.clean_reads/STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_q30.fastq.gz 
fastp v0.20.0, time used: 18 seconds
Number of reads after Q30 filtering for STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid:  6

### 10. Make BC count matrix

In [7]:
# Extract DNA sequences from fastq
cat STARRseq_names.txt | while read line
do
zcat 02.clean_reads/${line}_q30.fastq.gz | sed -n '2~4p' > 03.sequencing_saturation/${line}_q30.txt
done

In [8]:
# Generate BC count matrix for each samples
parallel \
"cat 03.sequencing_saturation/{}_q30.txt | sort | uniq -c | awk -F \" \" '{print \$2\"\t\"\$1}' \
   > 10.bc_count/{}_count.txt" ::: $(cat STARRseq_names.txt) 2>/dev/null

# Count number of unique BC per sample
cat STARRseq_names.txt | while read line
do
echo 10.bc_count/${line}_count.txt && cat 10.bc_count/${line}_count.txt | wc -l
done

10.bc_count/STA__073ce2__MM029_STARRseq_500_bp_enh_plasmid_count.txt
3861
10.bc_count/STA__1a9f59__MM087_STARRseq_500_bp_enh_plasmid_count.txt
3882
10.bc_count/STA__34085e__MM047_STARRseq_500_bp_enh_cDNA_count.txt
1516
10.bc_count/STA__3f00ec__MM001_STARRseq_500_bp_enh_cDNA_count.txt
921
10.bc_count/STA__4642d6__MM087_STARRseq_500_bp_enh_cDNA_count.txt
1914
10.bc_count/STA__60a828__MM047_STARRseq_500_bp_enh_plasmid_count.txt
3884
10.bc_count/STA__616867__MM029_STARRseq_500_bp_enh_cDNA_count.txt
2306
10.bc_count/STA__a6d28a__MM001_STARRseq_500_bp_enh_plasmid_count.txt
3174
10.bc_count/STA__af2595__MM074_STARRseq_500_bp_enh_cDNA_count.txt
1176
10.bc_count/STA__c67b3a__MM057_STARRseq_500_bp_enh_plasmid_count.txt
3903
10.bc_count/STA__e17937__MM074_STARRseq_500_bp_enh_plasmid_count.txt
3711
10.bc_count/STA__ed22cd__MM057_STARRseq_500_bp_enh_cDNA_count.txt
1974


In [17]:
# Assign BC to enhancers
parallel \
"awk -F '\t' -v OFS='\t' 'FNR==NR{a[\$2]=\$1 FS \$2;next}{ print a[\$1],\$2}' \
   <( cat Enh-freeBC.tsv |  sed 's/\t./\t/') \
   <( cat 10.bc_count/{}_count.txt ) \
   | awk -F '\t' '{if (\$1) print \$0;}' \
   > 10.bc_count/{}_count_final.txt" ::: $(cat STARRseq_names.txt) 2>/dev/null

In [19]:
cat 10.bc_count/STA__6541c2__MM099_STARRseq_500_bp_plasmid_count_final.txt | wc -l

45


In [20]:
cat 10.bc_count/STA__fc62b1__MM099_STARRseq_500_bp_cDNA_count_final.txt | wc -l

45


In [14]:
head 10.bc_count/STA__6541c2__MM099_STARRseq_500_bp_plasmid_count_final.txt


GPR39_2	AACAC	32604
SOX10_5A	AACCT	24319
COL5A1_3	AAGAC	37687
EDN1_1	AAGGT	31853
SERPINE1_1B	AATCG	63405
FOSL2_1A	ACAAT	20783
TYR_1	ACAGG	38207
SOX10_1B	ACAGT	52004
COL5A1_8	ACCAT	36634
BLK_5	ACCTA	27501


In [16]:
cat Enh-freeBC.tsv |  sed 's/\t./\t/'


FOSL2_1A	CAACAAT
FOSL2_1B	CGTATGG
SERPINE1_1A	AGGACAA
SERPINE1_1B	AGAATCG
SERPINE1_6	ATACTGT
FGF2_1A	CACGACT
FGF2_1B	CGTCGGT
FGF2_2	TACGTCC
FLNC_1A	GGTGTCT
FLNC_1B	TTAGCCA
HEG1_1	GACGCTT
BLK_3	GTGCCAA
BLK_5	ACACCTA
COL1A2_1	CAGTGTG
COL5A1_1	AGAGGTT
COL5A1_2	TCAGTGG
COL5A1_3	CTAAGAC
COL5A1_4	GGTTATC
COL5A1_5	GGTGTGG
COL5A1_8	CGACCAT
ALPK_2	TTCGCTC
AXL_2	GCGGAAT
ABCC3_1	ATGCCTT
GPR39_1	GAACTGG
GPR39_2	ACAACAC
EDN1_1	ACAAGGT
EDN1_3	GCCTATA
CDH1_1	GTAGCTT
CDH1_2	CATTCAT
SOX10_1A	CAACTTC
SOX10_1B	CCACAGT
SOX10_3A	TGTTCAC
SOX10_3B	CAAGCGT
SOX10_5A	AGAACCT
SOX10_5B	GTAGTAT
IRF4_1	TCCATGA
KIT_1A	TATCACG
KIT_1B	GATTGCT
MLANA_1	GAGTCTT
SGCD_2A	AGCTATT
SGCD_2B	ACACTGA
SGCD_3	TCTATCC
MITF_1	CTACTCC
TYR_1	CAACAGG
TYR_2	AAGAAGA